In [125]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.metrics import log_loss
from scipy.optimize import minimize

In [136]:
df = pd.read_csv('mytype.csv')

In [137]:
df.head()

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,is_iceberg,lr9,id
0,0.001405,0.007866,0.028444,0.000733,0.001917,0.009039,0,0.006688,dfd5f913
1,0.000000,0.000000,0.000206,0.000000,0.000046,0.000006,0,0.006576,e25388fd
2,0.859876,0.858316,0.833085,0.962498,0.996452,0.983096,1,0.977436,58b2aaa0
3,0.000337,0.003162,0.002622,0.000008,0.001263,0.000083,0,0.006679,4cfc3a18
4,0.006346,0.000255,0.017979,0.121713,0.046099,0.096352,0,0.006637,271f93f4


In [138]:
df['is_iceberg_max'] = df.iloc[:, :6].max(axis=1)
df['is_iceberg_min'] = df.iloc[:, :6].min(axis=1)
df['is_iceberg_mean'] = df.iloc[:, :6].mean(axis=1)
df['is_iceberg_median'] = df.iloc[:, :6].median(axis=1)

In [148]:

def get_minmax(df):
    a=np.where(np.sum((df.iloc[:,:6]>.5).astype(int),axis=1)>3, 
                                    df['is_iceberg_max'], 
                                    df['is_iceberg_min'])
    
    return a

def get_mean(df):
    a=df.iloc[:,:6].mean(axis=1) 
    return a
                                
    
def get_median(df):
    a=df.iloc[:,:6].mean(axis=1) 
    return a

def mae_func(weights,Y_values,predictions):
    ''' scipy minimize will pass the weights as a numpy array '''
    final_prediction = 0
    for weight, prediction in zip(weights, predictions):
            final_prediction += weight*prediction
    #print(final_prediction.shape)
    #print(prediction.shape)
    #print(Y_values.shape)
    return log_loss(Y_values, final_prediction)

def get_mini(df):
    predictions=[]
    Y_values = df['is_iceberg'].values
    #print(Y_values.shape) 
    lls = []
    wghts = []

    for i in range(6):
        predictions.append(np.array(df.iloc[:,i]))
  
    for i in range(100):
        starting_values = np.random.uniform(size=6)
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        bounds = [(0,1)]*len(predictions)
   
        res = minimize(fun=mae_func, x0=starting_values,args=(Y_values, predictions), method='L-BFGS-B', bounds=bounds, options={'disp': False, 'maxiter': 100000})

        lls.append(res['fun'])
        wghts.append(res['x'])
    # Uncomment the next line if you want to see the weights and scores calculated in real time
    #    print('Weights: {weights}  Score: {score}'.format(weights=res['x'], score=res['fun']))

    bestSC = np.min(lls)
    bestWght = wghts[np.argmin(lls)]
    #print(bestSC)
    print(bestWght)
    a = 0.0*Y_values
    for weight, prediction in zip(bestWght, predictions):
        a += weight*prediction
    return a

In [164]:
def gen_type(df):
    hig = .8
    mid = .5
    low = .2
 
    if df[0]>mid and df[1]>mid and df[2]>mid and df[3]>mid and df[4]>mid and df[5]>mid:
        return 1
    if df[0]<mid and df[1]<mid and df[2]<mid and df[3]<mid and df[4]<mid and df[5]<mid:
        return 1

    if df[0]>hig and df[1]>hig and df[2]>hig and df[3]>low and df[4]>low and df[5]>low:
        return 4
    if df[0]<low and df[1]<low and df[2]<low and df[3]<hig and df[4]<hig and df[5]<hig:
        return 4
    
    if df[0]>low and df[1]>low and df[2]>low and df[3]>hig and df[4]>hig and df[5]>hig:
        return 5
    if df[0]<hig and df[1]<hig and df[2]<hig and df[3]<low and df[4]<low and df[5]<low:
        return 5
    
    if df[0]>hig and df[1]>hig and df[2]>hig and df[3]<low and df[4]<low and df[5]<low:
        return 6
    if df[0]<low and df[1]<low and df[2]<low and df[3]>hig and df[4]>hig and df[5]>hig:
        return 6
    # 6 situation have 0 items

    

    if df[0]>mid and df[1]>mid and df[2]>mid and df[3]<low and df[4]<low and df[5]<low:
        return 7
    if df[0]<mid and df[1]<mid and df[2]<mid and df[3]>hig and df[4]>hig and df[5]>hig:
        return 7 
    # 7 situation have 2 items
    
    if np.sum((df[:6]>mid).astype(int))==5:
        return 8
    if np.sum((df[:6]<mid).astype(int))==5:
        return 8
    
    if np.sum((df[:6]>mid).astype(int))==4 and np.sum((df[:6]<low).astype(int))==0:
        return 9
    if np.sum((df[:6]<mid).astype(int))==5 and np.sum((df[:6]>hig).astype(int))==0:
        return 9
    return 0

In [165]:
df['type'] = df.apply(gen_type,1)

In [156]:
def test(type):
    df['new_pred'] = df['lr9']
    #a = df#[df.type==3]
    a = df[df.type==type]
    index = a.index
    print(a.shape)
    print('orig logloss is: ',log_loss(df['is_iceberg'], df['lr9']))
    pred = get_minmax(a)
    df.loc[index,'new_pred'] = pred
    print('minmax logloss is: ',log_loss(df['is_iceberg'],df['new_pred']))
    df['new_pred'] = df['lr9']
    #print('minmax logloss is: ',log_loss(df['is_iceberg'],df['new_pred']))

    pred = get_mean(a)
    df.loc[index,'new_pred'] = pred
    print('mean logloss is: ',log_loss(df['is_iceberg'],df['new_pred']))
    df['new_pred'] = df['lr9']
    #print('mean logloss is: ',log_loss(df['is_iceberg'],df['new_pred']))

    pred = get_median(a)
    df.loc[index,'new_pred'] = pred
    print('median logloss is: ',log_loss(df['is_iceberg'],df['new_pred']))
    df['new_pred'] = df['lr9']
    #print('median logloss is: ',log_loss(df['is_iceberg'],df['new_pred']))

    pred = get_mini(a)
    df.loc[index,'new_pred'] = pred
    print('mini logloss is: ',log_loss(df['is_iceberg'],df['new_pred']))
    df['new_pred'] = df['lr9']
    #print('mini logloss is: ',log_loss(df['is_iceberg'],df['new_pred']))

In [166]:
test(1)

(1343, 15)
orig logloss is:  0.125017134156
minmax logloss is:  0.122246526989
mean logloss is:  0.135284915428
median logloss is:  0.135284915428
[ 0.          0.07486111  0.09269978  0.00692075  0.8367096   0.        ]
mini logloss is:  0.126524402508


In [150]:
test(2)

(128, 15)
orig logloss is:  0.125017134156
minmax logloss is:  0.124213015331
mean logloss is:  0.125171129926
median logloss is:  0.125171129926
[ 0.          0.79142317  0.20743434  0.          0.01274252  0.        ]
mini logloss is:  0.122458147254


In [151]:
test(3)

(165, 15)
orig logloss is:  0.125017134156
minmax logloss is:  0.124074583051
mean logloss is:  0.133990313601
median logloss is:  0.133990313601
[ 0.          0.          0.          0.21153597  0.6174096   0.21023587]
mini logloss is:  0.128024497107


In [152]:
test(4)

(28, 15)
orig logloss is:  0.125017134156
minmax logloss is:  0.124491534557
mean logloss is:  0.128184907851
median logloss is:  0.128184907851
[ 0.          0.13039623  1.          0.          0.          0.        ]
mini logloss is:  0.124484618337


In [153]:
test(5)

(23, 15)
orig logloss is:  0.125017134156
minmax logloss is:  0.13027144279
mean logloss is:  0.124882148509
median logloss is:  0.124882148509
[ 0.          0.03686626  0.2731601   0.          0.85303185  0.        ]
mini logloss is:  0.123831955828


In [159]:
test(8)

(91, 15)
orig logloss is:  0.125017134156
minmax logloss is:  0.124689905769
mean logloss is:  0.130305618425
median logloss is:  0.130305618425
[ 0.01979893  0.0315118   0.60118082  0.05284784  0.31978043  0.02458192]
mini logloss is:  0.128940119718


In [160]:
test(9)

(22, 15)
orig logloss is:  0.125017134156
minmax logloss is:  0.129947948597
mean logloss is:  0.124668250722
median logloss is:  0.124668250722
[ 0.          1.          0.          0.          0.          0.15724953]
mini logloss is:  0.122695126354


In [42]:
a = df[(df.type==30) | (df.type==31)]
print(a.shape)
x,y = get_minmax(a)
print('orig logloss is: ',x)
print('now logloss is: ',y)

(128, 13)
orig logloss is:  0.21121289738
now logloss is:  0.201081586956


In [43]:
a = df[(df.type==50) | (df.type==51)]
print(a.shape)
x,y = get_minmax(a)
print('orig logloss is: ',x)
print('now logloss is: ',y)

(165, 13)
orig logloss is:  0.144646282663
now logloss is:  0.135462946777


In [44]:
a = df[(df.type==40) | (df.type==41)]
print(a.shape)
x,y = get_minmax(a)
print('orig logloss is: ',x)
print('now logloss is: ',y)

(28, 13)
orig logloss is:  0.0502674913562
now logloss is:  0.0201485478934


In [47]:
a = df[(df.type==70) | (df.type==71)]
print(a.shape)
x,y = get_minmax(a)
print('orig logloss is: ',x)
print('now logloss is: ',y)

(106, 13)
orig logloss is:  0.357931261918
now logloss is:  0.419475619422


In [48]:
a = df[(df.type==80) | (df.type==81)]
print(a.shape)
x,y = get_minmax(a)
print('orig logloss is: ',x)
print('now logloss is: ',y)

(25, 13)
orig logloss is:  0.653989037394
now logloss is:  1.01675867677


In [49]:
a = df[(df.type==60) | (df.type==61)]
print(a.shape)
x,y = get_minmax(a)
print('orig logloss is: ',x)
print('now logloss is: ',y)

(2, 13)


ValueError: y_true contains only one label (0.0). Please provide the true labels explicitly through the labels argument.

In [161]:
df[(df.lr9>.8)&(df.is_iceberg<.5)]

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,is_iceberg,lr9,id,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,type,new_pred
161,0.992831,0.964832,0.971465,0.998016,0.984880,0.997456,0,0.988721,2f881d78,0.998016,0.964832,0.984913,0.988855,1,0.988721
333,0.937305,0.925763,0.921871,0.770765,0.678706,0.644751,0,0.948894,74c92938,0.937305,0.644751,0.813194,0.846318,2,0.948894
350,0.978347,0.934119,0.903692,0.734713,0.917766,0.585872,0,0.965975,1303a495,0.978347,0.585872,0.842418,0.910729,2,0.965975
364,0.928160,0.829317,0.961569,0.693024,0.932323,0.752311,0,0.965776,5a501d33,0.961569,0.693024,0.849451,0.878738,2,0.965776
562,0.835171,0.844869,0.638103,0.502649,0.408982,0.897141,0,0.834270,55a724b1,0.897141,0.408982,0.687819,0.736637,8,0.834270
651,0.375417,0.437338,0.722559,0.522890,0.938852,0.868659,0,0.826154,f613255d,0.938852,0.375417,0.644286,0.622725,9,0.826154
661,0.901814,0.702368,0.348486,0.756717,0.789169,0.725438,0,0.845694,786ee92d,0.901814,0.348486,0.703999,0.741078,8,0.845694
684,0.838909,0.896258,0.949342,0.719365,0.831304,0.802169,0,0.967339,fd1f0c9b,0.949342,0.719365,0.839558,0.835106,2,0.967339
791,0.961608,0.973652,0.781880,0.914637,0.985917,0.839045,0,0.976749,93beaaaa,0.985917,0.781880,0.909457,0.938122,3,0.976749
795,0.793969,0.978606,0.969772,0.713314,0.902842,0.953195,0,0.977424,e8200704,0.978606,0.713314,0.885283,0.928019,3,0.977424


In [163]:
df[(df.lr9<.2)&(df.is_iceberg>.5)]

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,is_iceberg,lr9,id,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,type,new_pred
26,0.163231,0.131698,0.054178,0.980769,0.402647,0.777715,1,0.103521,958d42a8,0.980769,0.054178,0.418373,0.282939,0,0.103521
178,0.155006,0.049144,0.122662,0.953943,0.229853,0.096160,1,0.037888,a9ab128c,0.953943,0.049144,0.267795,0.138834,8,0.037888
276,0.668055,0.584428,0.431836,0.101356,0.114832,0.317042,1,0.180001,e0323d3d,0.668055,0.101356,0.369592,0.374439,0,0.180001
292,0.395070,0.110221,0.538535,0.038943,0.122276,0.033113,1,0.042504,a996191d,0.538535,0.033113,0.206360,0.116249,5,0.042504
308,0.037274,0.036769,0.020798,0.032937,0.030173,0.015020,1,0.008056,87319e1c,0.037274,0.015020,0.028829,0.031555,1,0.008056
370,0.337595,0.636648,0.629657,0.169332,0.113514,0.095298,1,0.198777,2d348f42,0.636648,0.095298,0.330341,0.253463,5,0.198777
462,0.088885,0.133706,0.624119,0.179305,0.377087,0.169005,1,0.075585,3c98959e,0.624119,0.088885,0.262018,0.174155,8,0.075585
743,0.316417,0.446939,0.302408,0.251646,0.502145,0.478173,1,0.197937,b7ca0494,0.502145,0.251646,0.382955,0.381678,8,0.197937
753,0.057107,0.006504,0.021739,0.003001,0.054998,0.005626,1,0.005647,3b431e27,0.057107,0.003001,0.024829,0.014121,1,0.005647
861,0.283816,0.309681,0.587182,0.000031,0.083383,0.005549,1,0.068291,e1b69ab4,0.587182,0.000031,0.211607,0.183599,5,0.068291


In [171]:
df[((df.is_iceberg_max-df.is_iceberg_min)>.75) & (df.lr9<.2) & (df.is_iceberg_mean>.4)]

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,is_iceberg,lr9,id,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,type,new_pred
26,0.163231,0.131698,0.054178,0.980769,0.402647,0.777715,1,0.103521,958d42a8,0.980769,0.054178,0.418373,0.282939,0,0.103521


In [172]:
df[((df.is_iceberg_max-df.is_iceberg_min)>.75) & (df.lr9>.8) & (df.is_iceberg_mean<.6)]

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,is_iceberg,lr9,id,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,type,new_pred


In [ ]:
from sklearn.preprocessing import MinMaxScaler
blend = np.concatenate([X_train,X_test])
scaling = MinMaxScaler(feature_range=(-1,1)).fit(blend)
blend = scaling.transform(blend)
X_train = blend[:X_train.shape[0],:]
X_test = blend[X_train.shape[0]:,:]